<a href="https://colab.research.google.com/github/edsondamasceno/deep-learning-methods/blob/main/Untrained_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";  # The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="4";  # Do other imports now...

# Billiotecas

In [ ]:
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import keras
import numpy as np
from sklearn.metrics import confusion_matrix ,accuracy_score,cohen_kappa_score,roc_curve,auc,recall_score,precision_score,f1_score
from keras.models import load_model

Using TensorFlow backend.


# Base

In [ ]:
train_path = '/home/edson/Documentos/Jupyter_Notebook/CIAR2018/train'
test_path = '/home/edson/Documentos/Jupyter_Notebook/CIAR2018/test'
val_path = '/home/edson/Documentos/Jupyter_Notebook/CIAR2018/val'
num_classes = 4
#Caminho das dados e quantidade de classes
#veja que dentro da pasta Train existe uma separação por pastas contendo as classes, e isso para tpdas as outras.
#exemplo '../_V2-GS1/Train/Glaucoma' '../_V2-GS1/Train/Normal'

In [ ]:
train_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(train_path, target_size=(224,224),color_mode='rgb',batch_size=12,shuffle=True)
test_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path, target_size=(224,224),color_mode='rgb',batch_size=12,shuffle=True)
val_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path, target_size=(224,224),color_mode='rgb',batch_size=12,shuffle=True)
#aqui vc vai carregar uma referencia de todo o conjunto de dados separados por bathcs

Found 280 images belonging to 4 classes.
Found 80 images belonging to 4 classes.
Found 40 images belonging to 4 classes.


In [ ]:
model = keras.applications.vgg16.VGG16(weights=None, include_top=True)
model.layers.pop()
new_layer = Dense(num_classes, activation='softmax', name='predictions')
model = Model(model.input, new_layer(model.layers[-1].output))
#aqui é bem simples, é apenas a criação do modelo, já com os pessos carregados da imagenet

In [ ]:
adam = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])
#configurações do modelo

In [ ]:
history = model.fit_generator(train_batches,epochs=300,validation_data=val_batches,verbose=1) #nessa linha vc faz o afinamento
model.save('VGG16.h5')#e aqui vc salva seu modelo, caso precise predizer em outra maquina, ou um outro dia.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
val_accuracy = history.history['val_accuracy']
val_loss = history.history['val_loss']

accuracy = history.history['accuracy']
loss = history.history['loss']

epochs = range(1, len(accuracy) + 1)

plt.figure(figsize=(12,8))
plt.plot(epochs, val_accuracy, 'b',color='blue', label='Val Accuracy')
plt.plot(epochs, accuracy, 'b', color='red',label='Training Accuracy' )
plt.title('Accuracy VGG16')
plt.legend()

plt.savefig('Acc_VGG16.svg')

plt.figure(figsize=(12,8))
plt.plot(epochs, val_loss, 'bo',color='blue', label='Val Loss')
plt.plot(epochs, loss, 'bo', color='red',label='Training Loss' )
plt.title('Loss VGG16')
plt.legend()

plt.savefig('loss_VGG16.svg')


import pandas as p
hist_df = pd.DataFrame(history.history)
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

#aqui é a parte que salva os graficos e o historico do afinamento

# Predição

In [ ]:
predicts_vectors = model.predict_generator(test_batches, verbose=1)

7/7 [==============================] - 29s 4s/step


In [ ]:
pred = np.argmax(predicts_vectors,axis=1)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer ()

lb.fit(test_batches.labels)

y_test = lb.transform (test_batches.labels)
y_pred = lb.transform (pred)

auc_score = roc_auc_score(y_test, y_pred,average="macro")

In [ ]:
y_true = test_batches.labels
y_pred = pred

recall = recall_score(y_true, y_pred, average='macro')
precision = precision_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
kappa = cohen_kappa_score(test_batches.labels, pred)
acc = accuracy_score(test_batches.labels, pred)

/usr/local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print('>> Resultados <<')
print('Acc: {}'.format(acc*100))
print('Recall: {}'.format(recall*100))
print('Precision: {}'.format(precision*100))
print('F1-Acore: {}'.format(f1))
print('AUC: {}'.format(auc_score))
print('Kappa: {}'.format(kappa))

In [ ]:
confusion_matrix(y_true,y_pred)